## 数量化1類

In [3]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import japanize_matplotlib
from sklearn.linear_model import LinearRegression

小〜高校生の男女の身長のデータを数量化1類で分析してみる

- 身長のデータは政府統計から取って加工している
    - https://www.e-stat.go.jp/stat-search/files?page=1&layout=datalist&toukei=00400002&tstat=000001011648&cycle=0&tclass1=000001138504&tclass2=000001138505&tclass3val=0
- ↑から平均と標準偏差を計算し，再サンプルしているため負の身長や3m超えの身長のデータが存在する
    - あくまで人工データとして利用する

In [4]:
df = pd.read_csv('../data/qualitization_height.csv')

display(df.head())
display(df.describe())

,id,学校,性別,身長
0,0,小,男,106.875681
1,1,小,男,159.686501
2,2,小,男,110.676980
3,3,小,男,113.834546
4,4,小,男,130.482813


,id,身長
count,2400.000000,2400.000000
mean,1199.500000,145.360599
std,692.964646,40.646600
min,0.000000,-1.736570
25%,599.750000,118.143479
50%,1199.500000,143.736185
75%,1799.250000,171.437828
max,2399.000000,303.257805


### data overview

### modeling

- 概要
    - 数量化1類は，カテゴリ変数を説明変数$X$として目的変数$y$への影響を調べる分析手法
    - やっていることは，onehot-vector変数を用いた重回帰分析


- 発想
    - データ$a_{小}, a_{中}, a_{高}, a_{男}, a_{女} \in \mathbb{R}$を観測したとき，小学生男子の身長 $\fallingdotseq a_{小} + a_{男}$, 高校生女子の身長$\fallingdotseq a_{高} + a_{女}$のような近似がうまくいっていれば，この変数$a$らは，｢小学生である｣ことや，｢女子である｣ことと身長の関係性をよく表現できていると考えられる
    - 身長という尺度を通して，カテゴリ変数を数値に変換している(これが｢数量化｣と呼ばれる所以)

- 方法
    - モチベーションを理解した上でどのように実現するのか？
        - 近似の精度はどうやって評価するのか？
        - いい感じの$a$を見つける方法は？
    - 数学的理解
        - $y$の特徴をaたちがよく表しているとは，$y$の分散$Var[y]$とMSE=$\frac{1}{N}\sum(y_i - a_{.})^2$の大小関係で評価
        - 比$\frac{MSE}{Var[y]}$が小さいとき，近似がうまくいっていると考える
        - **これはone-hot vector $x_i$を導入したとき，$y_i = \bold{a_i}\bold{X_i}$のMSEを最小化する最小二乗法と完全に一致**
        - 実際には，モデル$y = a. + b$を考える
            - $b$は，身長に関する全員に共通した特徴を示す定数項
            - 得られた$a$のパターンは同じ個体でも様々な組み合わせが存在する
                - 解釈しやすいものや得られたものを使用すれば良い


まずは全ての変数を利用して学習してみる

In [12]:
y = df['身長']
X = pd.get_dummies(df[['学校', '性別']])
display(X.head())

,学校_中,学校_小,学校_高,性別_女,性別_男
0,0,1,0,0,1
1,0,1,0,0,1
2,0,1,0,0,1
3,0,1,0,0,1
4,0,1,0,0,1


In [13]:
# 学習
base_model = LinearRegression()
base_model.fit(X, y)

LinearRegression()

In [14]:
print(f"coef: {base_model.coef_}")
print(f"intercept: {base_model.intercept_}")

coef: [ 1.14957822e+14  1.14957822e+14  1.14957822e+14 -1.70908618e+14
 -1.70908618e+14]
intercept: 55950795525808.75


In [18]:
# なんか係数の値でかくて解釈に困る

変数を選んでやってみる

In [20]:
# 学校_小，性別_男を使わない
y = df['身長']
X = pd.get_dummies(df[['学校', '性別']])[['学校_中', '学校_高', '性別_女']]

# 学習
model = LinearRegression()
model.fit(X, y)

print(f"coef: {model.coef_}")
print(f"intercept: {model.intercept_}")
print(f"小学生男子の平均身長：{df[(df['学校'] == '小') & (df['性別'] == '男')]['身長'].mean()}")

coef: [24.53840691 32.02907977 -5.69432178]
intercept: 134.06588773831584
小学生男子の平均身長：132.16344891581667


- `学校_小`と`性別_男`を使用していないのでこれらの係数は0．それを踏まえると，得られた結果は以下

|$a_小$|$a_中$|$a_高$|$a_男$|$a_女$|$b$|
|---|---|---|---|---|---|
|0|24.54|32.03|0|-5.69|134.07|

- $a_小=0, a_男=0$より小学生男子基準で係数を解釈することになる
    - `学校_小`と`性別_男`のカラムを使用せずにロジスティック回帰モデルを学習
- 得られる考察
    - 小学生男子の平均身長は概ね134.07cmである(実際は132.16cmであるが，数値計算上での誤差がある)．
        - 小学生男子であることは係数bに含まれることになる？
            - なんで$b$と使わないカラムの平均が一致するのかあんまり理解できていない...
    - 中学生であることは，身長という尺度で考えたときに，24.54cm高いということを意味する．
    - 女性であるということは，身長という尺度で考えたときに，-5.69cm小さいということを意味する．

In [22]:
# 
X_only_school = X.copy()
X_only_school['性別_女'] = 0
# a_女を使用せずに身長を予測?
# その結果から小学生男子の平均身長を引いている
bias_school = model.predict(X_only_school).mean() - model.intercept_
# これにマイナスをかけたものが小学生であることの身長への影響→なんで？
print(bias_school)

14.141871669682502


In [23]:
X_only_school.head()

,学校_中,学校_高,性別_女
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [21]:
X_only_sex = X.copy()
X_only_sex[['学校_中', '学校_高']] = 0

bias_sex = model.predict(X_only_sex).mean() - model.intercept_

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 5 is different from 3)